In [39]:
import pandas as pd
import json
games = pd.read_csv('games.csv')
recommendations = pd.read_csv('recommendations.csv')
users = pd.read_csv('users.csv')

print(f'Games:{len(games)}')
print(f'Recommendations:{len(recommendations)}')
print(f'Users:{len(users)}')

Games:50872
Recommendations:41154794
Users:14306064


In [40]:
from tqdm import tqdm
"""Carica le descrizioni dei giochi da games_metadata.json in un dizionario."""
metadata_dict = {}
print("Caricamento metadata dei giochi...")
try:
    with open("games_metadata.json", 'r', encoding='utf-8') as f:
        for line in tqdm(f, desc="Lettura metadata"):
            try:
                data = json.loads(line)
                if 'app_id' in data and 'description' in data:
                    metadata_dict[data['app_id']] = data['description']
            except json.JSONDecodeError:
                continue # Salta le linee malformate
except FileNotFoundError:
    print(f"ERRORE: File non trovato: ")

Caricamento metadata dei giochi...


Lettura metadata: 50872it [00:00, 310202.72it/s]


## Rimozione item con caratteri giappo/coreani

In [41]:
import json
import re

def filter_apps_by_language(file_path):
    """
    Legge un file contenente JSON per riga e filtra gli app_id
    con description o title in giapponese o coreano.
    """
    # Questo pattern regex include gli intervalli Unicode per:
    # Hiragana (Giapponese), Katakana (Giapponese),
    # CJK Unified Ideographs (Kanji/Hanja/Hanzi), e Hangul (Coreano).
    cjk_pattern = re.compile(
        r'[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF\uAC00-\uD7AF]'
    )

    found_app_ids = []

    print(f"Apertura del file: {file_path}\n")

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data = json.loads(line)
                app_id = data.get("app_id")
                
                # Combiniamo titolo e descrizione per controllarli insieme.
                # Usiamo .get() per evitare errori se un campo manca.
                title = data.get("title", "") or ""
                description = data.get("description", "") or ""
                text_to_check = title + description

                # re.search() è efficiente perché si ferma al primo match
                if cjk_pattern.search(text_to_check):
                    if app_id:
                        found_app_ids.append(app_id)
            
            except json.JSONDecodeError:
                print(f"Attenzione: saltata una riga non valida: {line.strip()}")

    return found_app_ids

# --- Come usarlo ---

# 1. Salva i tuoi dati in un file chiamato "apps.jsonl".
#    (Ogni oggetto JSON deve essere su una nuova riga).

# 2. Esegui la funzione e stampa i risultati.
problematic_ids = filter_apps_by_language("games_metadata.json")

print("Elaborazione completata.\n")
if problematic_ids:
    print(f"✅ Trovati {len(problematic_ids)} app_id con caratteri giapponesi/coreani/cinesi:")
    print(problematic_ids)
else:
    print("ℹ️ Nessun app_id corrispondente ai criteri è stato trovato.")

Apertura del file: games_metadata.json

Elaborazione completata.

✅ Trovati 787 app_id con caratteri giapponesi/coreani/cinesi:
[910320, 570770, 716710, 1310990, 1700220, 1066290, 1126310, 987480, 1503670, 1475430, 1536080, 2105980, 1294060, 1973610, 1571140, 1168430, 2185770, 1395850, 1228520, 2020530, 2111860, 1345740, 913160, 1536070, 2059810, 965820, 1457630, 2101110, 1879640, 1549240, 1744910, 1058770, 2020510, 1984350, 1084520, 1478410, 909380, 1121300, 1180320, 876850, 1168470, 1261220, 1164000, 948830, 1070020, 1401220, 1017410, 2258590, 1161190, 1900340, 1120370, 1437840, 1176050, 1649730, 2193880, 1130620, 1672670, 528060, 2097720, 779640, 929200, 1267310, 1559300, 899160, 1411330, 2020520, 738510, 1437850, 1597460, 1250770, 1025070, 1306750, 1546570, 2109640, 769550, 1043230, 1437870, 2155310, 1069230, 1611490, 745880, 1195830, 1315160, 1508750, 1424720, 1840540, 937570, 1266630, 342380, 982400, 983150, 1422540, 1374480, 2020480, 1189490, 1658880, 2115700, 1161930, 1437830, 

In [42]:
ids_descrizione = [
    app_id for app_id, description in metadata_dict.items() if description
]

# --- Verifica (opzionale) ---
# Stampa quanti ID hai trovato
print(f"Trovati {len(ids_descrizione)} giochi con descrizione.")

# Stampa i primi 20 ID per controllare
if ids_descrizione:
    print("Esempio dei primi 20 app_id trovati:")
    print(ids_descrizione[:20])

Trovati 40499 giochi con descrizione.
Esempio dei primi 20 app_id trovati:
[13500, 113020, 226560, 249050, 250180, 253980, 282900, 19810, 15270, 21130, 22130, 29180, 32750, 241620, 408520, 244910, 250460, 278890, 312200, 321290]


In [43]:
games_filtered = games[games['app_id'].isin(ids_descrizione)]
print(f'Righe salvate: {len(games_filtered)}')

Righe salvate: 40499


In [44]:
games_filtered = games_filtered[~games_filtered['app_id'].isin(problematic_ids)]
print(f'Righe salvate: {len(games_filtered)}')

Righe salvate: 39712


In [45]:
games_filtered.to_csv('filtering_no_desc_giappo_corean_k10/games.csv', index=False)

In [46]:
recommendations['app_id'].value_counts().min(), recommendations['app_id'].value_counts().max()

(np.int64(1), np.int64(319492))

In [47]:
MIN_GAME_INTERACTIONS = 10

## 1

In [54]:
len(problematic_ids)

787

In [53]:
for i in problematic_ids:
    if i == 910320:
        print(f"Skipping problematic app_id: {i}")
        continue

Skipping problematic app_id: 910320


In [ ]:
# Filtra users per gli user_id presenti in new_recommendations
print(f"Righe prima del filtro: {len(recommendations)}")
recommendations = recommendations[recommendations['app_id'].isin(ids_descrizione)]
print(f"Righe dopo il filtro: {len(recommendations)}")

Righe prima del filtro: 41154794
Righe dopo il filtro: 20347591


In [63]:
recommendations['app_id'] = recommendations['app_id'].astype('Int64')

In [68]:
problematic_ids_int = [int(pid) for pid in problematic_ids]
print(f"Righe prima del filtro problematici: {len(recommendations)}")
recommendations = recommendations[~recommendations['app_id'].isin(problematic_ids)]
print(f"Righe dopo il filtro problematici: {len(recommendations)}")

Righe prima del filtro problematici: 20347591
Righe dopo il filtro problematici: 20182554


In [70]:
print(f"-> Prima filtraggio giochi: {recommendations['app_id'].unique().size} righe rimaste")
print(f"-> Prima filtraggio utenti: {recommendations['user_id'].unique().size} righe rimaste")
    
# Filtra i giochi con poche interazioni
game_counts = recommendations['app_id'].value_counts()
games_to_keep = game_counts[game_counts >= MIN_GAME_INTERACTIONS].index
df_filtered_games = recommendations[recommendations['app_id'].isin(games_to_keep)]


# Filtra gli utenti con poche interazioni
user_counts = df_filtered_games['user_id'].value_counts()
users_to_keep = user_counts[user_counts >= MIN_GAME_INTERACTIONS].index
new_recommendations = df_filtered_games[df_filtered_games['user_id'].isin(users_to_keep)]

print(f"-> Dopo filtraggio giochi: {new_recommendations['app_id'].unique().size} righe rimaste")
print(f"-> Dopo filtraggio utenti: {new_recommendations['user_id'].unique().size} righe rimaste")

-> Prima filtraggio giochi: 36314 righe rimaste
-> Prima filtraggio utenti: 7449206 righe rimaste
-> Dopo filtraggio giochi: 30061 righe rimaste
-> Dopo filtraggio utenti: 296410 righe rimaste


## 2

In [71]:
print(f"-> Prima filtraggio giochi: {new_recommendations['app_id'].unique().size} righe rimaste")
print(f"-> Prima filtraggio utenti: {new_recommendations['user_id'].unique().size} righe rimaste")
    
# Filtra i giochi con poche interazioni
game_counts = new_recommendations['app_id'].value_counts()
games_to_keep = game_counts[game_counts >= MIN_GAME_INTERACTIONS].index
df_filtered_games = new_recommendations[new_recommendations['app_id'].isin(games_to_keep)]


# Filtra gli utenti con poche interazioni
user_counts = df_filtered_games['user_id'].value_counts()
users_to_keep = user_counts[user_counts >= MIN_GAME_INTERACTIONS].index
new_recommendations = df_filtered_games[df_filtered_games['user_id'].isin(users_to_keep)]

print(f"-> Dopo filtraggio giochi: {new_recommendations['app_id'].unique().size} righe rimaste")
print(f"-> Dopo filtraggio utenti: {new_recommendations['user_id'].unique().size} righe rimaste")

-> Prima filtraggio giochi: 30061 righe rimaste
-> Prima filtraggio utenti: 296410 righe rimaste
-> Dopo filtraggio giochi: 25885 righe rimaste
-> Dopo filtraggio utenti: 295417 righe rimaste


## 3

In [72]:
print(f"-> Prima filtraggio giochi: {new_recommendations['app_id'].unique().size} righe rimaste")
print(f"-> Prima filtraggio utenti: {new_recommendations['user_id'].unique().size} righe rimaste")
    
# Filtra i giochi con poche interazioni
game_counts = new_recommendations['app_id'].value_counts()
games_to_keep = game_counts[game_counts >= MIN_GAME_INTERACTIONS].index
df_filtered_games = new_recommendations[new_recommendations['app_id'].isin(games_to_keep)]


# Filtra gli utenti con poche interazioni
user_counts = df_filtered_games['user_id'].value_counts()
users_to_keep = user_counts[user_counts >= MIN_GAME_INTERACTIONS].index
new_recommendations = df_filtered_games[df_filtered_games['user_id'].isin(users_to_keep)]

print(f"-> Dopo filtraggio giochi: {new_recommendations['app_id'].unique().size} righe rimaste")
print(f"-> Dopo filtraggio utenti: {new_recommendations['user_id'].unique().size} righe rimaste")

-> Prima filtraggio giochi: 25885 righe rimaste
-> Prima filtraggio utenti: 295417 righe rimaste
-> Dopo filtraggio giochi: 25846 righe rimaste
-> Dopo filtraggio utenti: 295399 righe rimaste


## 4

In [73]:
print(f"-> Prima filtraggio giochi: {new_recommendations['app_id'].unique().size} righe rimaste")
print(f"-> Prima filtraggio utenti: {new_recommendations['user_id'].unique().size} righe rimaste")
    
# Filtra i giochi con poche interazioni
game_counts = new_recommendations['app_id'].value_counts()
games_to_keep = game_counts[game_counts >= MIN_GAME_INTERACTIONS].index
df_filtered_games = new_recommendations[new_recommendations['app_id'].isin(games_to_keep)]


# Filtra gli utenti con poche interazioni
user_counts = df_filtered_games['user_id'].value_counts()
users_to_keep = user_counts[user_counts >= MIN_GAME_INTERACTIONS].index
new_recommendations = df_filtered_games[df_filtered_games['user_id'].isin(users_to_keep)]

print(f"-> Dopo filtraggio giochi: {new_recommendations['app_id'].unique().size} righe rimaste")
print(f"-> Dopo filtraggio utenti: {new_recommendations['user_id'].unique().size} righe rimaste")

-> Prima filtraggio giochi: 25846 righe rimaste
-> Prima filtraggio utenti: 295399 righe rimaste
-> Dopo filtraggio giochi: 25841 righe rimaste
-> Dopo filtraggio utenti: 295398 righe rimaste


In [74]:
print(f"-> Prima filtraggio giochi: {new_recommendations['app_id'].unique().size} righe rimaste")
print(f"-> Prima filtraggio utenti: {new_recommendations['user_id'].unique().size} righe rimaste")
    
# Filtra i giochi con poche interazioni
game_counts = new_recommendations['app_id'].value_counts()
games_to_keep = game_counts[game_counts >= MIN_GAME_INTERACTIONS].index
df_filtered_games = new_recommendations[new_recommendations['app_id'].isin(games_to_keep)]


# Filtra gli utenti con poche interazioni
user_counts = df_filtered_games['user_id'].value_counts()
users_to_keep = user_counts[user_counts >= MIN_GAME_INTERACTIONS].index
new_recommendations = df_filtered_games[df_filtered_games['user_id'].isin(users_to_keep)]

print(f"-> Dopo filtraggio giochi: {new_recommendations['app_id'].unique().size} righe rimaste")
print(f"-> Dopo filtraggio utenti: {new_recommendations['user_id'].unique().size} righe rimaste")

-> Prima filtraggio giochi: 25841 righe rimaste
-> Prima filtraggio utenti: 295398 righe rimaste
-> Dopo filtraggio giochi: 25840 righe rimaste
-> Dopo filtraggio utenti: 295398 righe rimaste


In [75]:
print(f"-> Prima filtraggio giochi: {new_recommendations['app_id'].unique().size} righe rimaste")
print(f"-> Prima filtraggio utenti: {new_recommendations['user_id'].unique().size} righe rimaste")
    
# Filtra i giochi con poche interazioni
game_counts = new_recommendations['app_id'].value_counts()
games_to_keep = game_counts[game_counts >= MIN_GAME_INTERACTIONS].index
df_filtered_games = new_recommendations[new_recommendations['app_id'].isin(games_to_keep)]


# Filtra gli utenti con poche interazioni
user_counts = df_filtered_games['user_id'].value_counts()
users_to_keep = user_counts[user_counts >= MIN_GAME_INTERACTIONS].index
new_recommendations = df_filtered_games[df_filtered_games['user_id'].isin(users_to_keep)]

print(f"-> Dopo filtraggio giochi: {new_recommendations['app_id'].unique().size} righe rimaste")
print(f"-> Dopo filtraggio utenti: {new_recommendations['user_id'].unique().size} righe rimaste")

-> Prima filtraggio giochi: 25840 righe rimaste
-> Prima filtraggio utenti: 295398 righe rimaste
-> Dopo filtraggio giochi: 25840 righe rimaste
-> Dopo filtraggio utenti: 295398 righe rimaste


In [76]:
print(f"-> Giochi persi dopo il filtraggio: {recommendations['app_id'].unique().size - new_recommendations['app_id'].unique().size}")
print(f"-> Utenti persi dopo il filtraggio: {recommendations['user_id'].unique().size - new_recommendations['user_id'].unique().size}")

-> Giochi persi dopo il filtraggio: 10474
-> Utenti persi dopo il filtraggio: 7153808


In [77]:
new_recommendations

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
800,1544020,18,2,2022-12-02,False,9.1,9254726,800
954,1544020,0,0,2022-12-12,False,0.8,12303769,954
2090,1766740,0,0,2022-12-18,True,12.7,6766810,2090
2165,1325200,0,0,2021-06-10,True,94.5,11593837,2165
4636,1325200,0,0,2021-12-02,False,76.9,12938782,4636
...,...,...,...,...,...,...,...,...
41154770,700600,0,0,2021-07-14,True,2.0,6382076,41154770
41154771,362960,0,0,2019-07-06,True,35.0,7858713,41154771
41154777,298110,0,0,2022-03-25,True,5.0,13661350,41154777
41154790,758870,8,0,2019-07-18,False,8.0,1786254,41154790


In [78]:
new_recommendations.to_csv('filtering_no_desc_giappo_corean_k10/recommendations.csv', index=False)

In [79]:
# Filtra users per gli user_id presenti in new_recommendations
users_filtered = users[users['user_id'].isin(new_recommendations['user_id'])]

# Salva in CSV
users_filtered.to_csv('filtering_no_desc_giappo_corean_k10/users.csv', index=False)

print(f'Righe salvate: {len(users_filtered)}')

Righe salvate: 295398


In [80]:
from sklearn.model_selection import train_test_split

recommendations = pd.read_csv('filtering_no_desc_giappo_corean_k10/recommendations.csv')
train_recommendations, test_recommendations = train_test_split(recommendations, test_size=0.2, random_state=42)

In [82]:
len(train_recommendations), len(test_recommendations), len(recommendations)

(5421511, 1355378, 6776889)

In [83]:
train_recommendations.to_csv('filtering_no_desc_giappo_corean_k10/train_recommendations.csv', index=False)
test_recommendations.to_csv('filtering_no_desc_giappo_corean_k10/test_recommendations.csv', index=False)